## 加载模块

In [133]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
import dask
from dask_ml.preprocessing import DummyEncoder
from dask_ml.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from dask_glm.datasets import make_classification
import gc

In [134]:
gc.collect()

1841

## 加载数据

### 其它

In [2]:
path = 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format2\\train_format2.csv'
df_train2 = dd.read_csv(path,dtype={'age_range': 'float32','gender': 'float32'},
                        assume_missing = True)
# 如果age_range、或者gender指定为int类型的话，使用head方法将报错

In [3]:
df_train2.head(5)

,user_id,age_range,gender,merchant_id,label,activity_log
0,34176.0,6.0,0.0,944.0,-1.0,408895:1505:7370:1107:0
1,34176.0,6.0,0.0,412.0,-1.0,17235:1604:4396:0818:0#954723:1604:4396:0818:0...
2,34176.0,6.0,0.0,1945.0,-1.0,231901:662:2758:0818:0#231901:662:2758:0818:0#...
3,34176.0,6.0,0.0,4752.0,-1.0,174142:821:6938:1027:0
4,34176.0,6.0,0.0,643.0,-1.0,716371:1505:968:1024:3


In [4]:
df_train2.loc[df_train2.gender.isnull()].head()

,user_id,age_range,gender,merchant_id,label,activity_log
8269,226176.0,NaN,NaN,1992.0,-1.0,547751:1349:7279:0902:0
8270,226176.0,NaN,NaN,487.0,-1.0,817592:1349:7532:0624:0#894236:36:7532:0624:0#...
8271,226176.0,NaN,NaN,3910.0,-1.0,465321:1505:5484:0913:0
8272,226176.0,NaN,NaN,333.0,-1.0,647284:88:7621:0610:0
8273,226176.0,NaN,NaN,1383.0,-1.0,619906:656:3578:1104:0


In [5]:
path2 = 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format2\\test_format2.csv'

In [9]:
df_test2_n = dd.read_csv(path2, assume_missing=True )

In [10]:
df_test2_n.info()

<class 'dask.dataframe.core.DataFrame'>
Columns: 6 entries, user_id to activity_log
dtypes: object(1), float64(5)

In [11]:
df_test2_n.head(5)

,user_id,age_range,gender,merchant_id,label,activity_log
0,163968.0,0.0,0.0,4378.0,-1.0,101206:812:6968:0614:0
1,163968.0,0.0,0.0,2300.0,-1.0,588758:844:3833:0618:0#71782:844:3833:1111:2#7...
2,163968.0,0.0,0.0,1551.0,-1.0,312747:243:1954:0627:0#312747:243:1954:0627:0#...
3,163968.0,0.0,0.0,4343.0,-1.0,932390:1612:3201:0628:0
4,163968.0,0.0,0.0,4911.0,-1.0,957657:662:3089:0612:0


In [12]:
df_test2_n[df_test2_n['label'].isnull()].head(8)

,user_id,age_range,gender,merchant_id,label,activity_log
12,163968.0,0.0,0.0,4605.0,NaN,772645:1368:7622:1111:2#772645:1368:7622:1111:0
53,360576.0,2.0,2.0,1581.0,NaN,948181:614:4066:1111:2#1111020:614:4066:1111:2...
65,98688.0,6.0,0.0,1964.0,NaN,774035:247:1193:1111:0#774035:247:1193:1111:0#...
67,98688.0,6.0,0.0,3645.0,NaN,661340:748:3637:1111:0#661340:748:3637:1111:2#...
134,295296.0,2.0,1.0,3361.0,NaN,521280:1118:487:1111:0#521280:1118:487:1111:0#...
170,33408.0,2.0,0.0,98.0,NaN,1045198:1038:1208:1029:0#430454:1038:1208:1029...
228,230016.0,5.0,1.0,1742.0,NaN,962700:464:7885:1111:2#144958:464:7885:1111:0#...
242,164736.0,3.0,1.0,598.0,NaN,974463:1188:6983:1111:2#974463:1188:8351:1111:0


In [3]:
path3 = 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\test_format1.csv'

In [4]:
df_test1 = dd.read_csv(path3)
df_test1.sample(5)

,user_id,merchant_id,prob
npartitions=1,,,
,int64,int64,float64
,...,...,...


### train1数据

In [5]:
path4 = 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\train_format1.csv'

In [6]:
df_train1 = dd.read_csv(path4)

In [25]:
df_train1.head(5)

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0
2,34176,4356,1
3,34176,2217,0
4,230784,4818,0


In [27]:
df_train1[df_train1['label'].isnull()].compute()
# 上述生成的是空集

,user_id,merchant_id,label


### user_info数据

In [7]:
path5 = 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\user_info_format1.csv'

In [8]:
user_info = dd.read_csv(path5)

In [30]:
user_info.head()

,user_id,age_range,gender
0,376517,6.0,1.0
1,234512,5.0,0.0
2,344532,5.0,0.0
3,186135,5.0,0.0
4,30230,5.0,0.0


### 1.78G的user_log数据导入

In [10]:
path6 = 'E:\\MySQL_data\\repeat_buyers_prediction\\data_format1\\user_log_format1.csv'   

In [11]:
user_log_data = dd.read_csv(path6)

In [33]:
user_log_data.head(6)

,user_id,item_id,cat_id,seller_id,brand_id,time_stamp,action_type
0,328862,323294,833,2882,2661.0,829,0
1,328862,844400,1271,2882,2661.0,829,0
2,328862,575153,1271,2882,2661.0,829,0
3,328862,996875,1271,2882,2661.0,829,0
4,328862,1086186,1271,1253,1049.0,829,0
5,328862,623866,1271,2882,2661.0,829,0


## 处理user_log数据

### user_id被记录的信息条数 

In [12]:
u1 = user_log_data[['user_id']].copy()
u1 = u1.groupby('user_id').size().reset_index()
u1 = u1.rename(columns = {0:'user_action_count'})

In [35]:
u1.head()

,user_id,user_action_count
0,16,67
1,19,114
2,56,281
3,155,66
4,201,53


### 根据user_id对action_type编码

In [14]:
u3 = user_log_data[['user_id','action_type']].copy()

# set(u3.action_type.isnull()).compute()
# 结果为False，说明actiio_type中没有空值

u3 = u3.categorize(columns = ['action_type'])

u3 = DummyEncoder().fit_transform(u3)

In [39]:
u3.head()

,user_id,action_type_0,action_type_2,action_type_3,action_type_1
0,328862,1,0,0,0
1,328862,1,0,0,0
2,328862,1,0,0,0
3,328862,1,0,0,0
4,328862,1,0,0,0


### 对user_id操作的天数计数

In [15]:
u4 = user_log_data[['user_id','time_stamp']].copy()

u4 = u4[['user_id','time_stamp']].drop_duplicates()

u4 = u4.groupby(['user_id']).size().reset_index()

u4 = u4.rename(columns = {0:'user_handle_day_count'})

In [41]:
u4_h = u4.loc[:5,:].copy()
u4_h.compute() # 为什么这一步的时候那么的消耗内存呢？

,user_id,user_handle_day_count
0,1,5
1,2,9
2,3,13
3,4,10
4,5,30
5,6,56


### 每个user_id对每个item_id(具体的商品)的操作数量编码

In [16]:
u5 = user_log_data[['user_id','item_id','action_type']].copy()

u5 = u5.categorize(columns = ['action_type'])

u5 = DummyEncoder().fit_transform(u5)

u5 = u5.groupby(['user_id','item_id']).sum().reset_index()

In [46]:
u5_h = u5.loc[:4,:].copy() # 只是这里生成的行索引没有重排，不知道改怎么办！
u5_h.compute()

,user_id,item_id,action_type_0,action_type_2,action_type_3,action_type_1
0,16,51942,1.0,0.0,0.0,0.0
1,16,64543,6.0,1.0,0.0,0.0
2,16,83910,1.0,0.0,0.0,0.0
3,16,140170,1.0,0.0,0.0,0.0
4,16,159165,1.0,0.0,0.0,0.0


### 每个user_id对每个brand_id的操作编码

In [17]:
u6 = user_log_data[['user_id','brand_id','action_type']].copy()

u6 = u6.categorize(columns = ['action_type'])

u6 = DummyEncoder().fit_transform(u6)
u6 = u6.groupby(['user_id','brand_id']).sum().reset_index()

In [50]:
u6.loc[:3,:].compute()

,user_id,brand_id,action_type_0,action_type_2,action_type_3,action_type_1
0,16,277.0,1.0,0.0,0.0,0.0
1,16,625.0,1.0,0.0,0.0,0.0
2,16,654.0,2.0,0.0,0.0,0.0
3,16,672.0,4.0,0.0,0.0,0.0


### 每个user_id对每个seller_id下的item_id的action_type编码

In [18]:
u7 = user_log_data[['user_id','seller_id','item_id','action_type']].copy()
u7 = u7.rename(columns = {'action_type':'usi_action_type'})
u7 = u7.categorize(columns = ['usi_action_type'])
u7 = DummyEncoder().fit_transform(u7)
u7 = u7.groupby(['user_id','seller_id','item_id']).sum().reset_index()

In [55]:
u7.head()

,user_id,seller_id,item_id,usi_action_type_0,usi_action_type_2,usi_action_type_3,usi_action_type_1
0,16,574,197311,1.0,0.0,0.0,0.0
1,16,650,218217,1.0,0.0,0.0,0.0
2,16,650,293373,1.0,0.0,0.0,0.0
3,16,650,391126,1.0,0.0,0.0,0.0
4,16,650,673172,4.0,0.0,0.0,0.0


### 按照cat_id分类统计item_id的数量

In [19]:
i1 = user_log_data[['cat_id','item_id']].copy()
i1 = i1.drop_duplicates()
i1 = i1.groupby('cat_id').size().reset_index()

In [20]:
i1 = i1.rename( columns={0: 'cat_item_count'} ) # dask不支持参数inplace

In [20]:
i1_h = i1.loc[:5,:].copy() # 如果直接用i1.head()，那么笔记本8G内存就不够了。
i1_h.compute()

,cat_id,cat_item_count
0,1,34
1,2,2303
2,3,12
3,4,552
4,5,551
5,6,5


### 按照seller_id分类统计brand_id的数量

In [21]:
i2 = user_log_data[['seller_id','brand_id']].copy()
i2 = i2.drop_duplicates()
i2 = i2.groupby('seller_id').size().reset_index()
i2 = i2.rename( columns={0: 'seller_brand_count'} )

In [22]:
i2_h = i2.loc[:5,:].copy()
i2_h.compute()

,seller_id,seller_brand_count
0,1,3
1,2,2
2,3,2
3,4,3
4,5,2
5,6,3


### 按照seller_id统计item_id的个数

In [22]:
i3 = user_log_data[['seller_id','item_id']].copy()
i3 = i3.drop_duplicates()
i3 = i3.groupby('seller_id').size().reset_index()
i3 = i3.rename(columns = {0:'seller_item_id_count'})

In [62]:
i3.head()

,seller_id,seller_item_id_count
0,1,2977
1,2,154
2,3,171
3,4,155
4,5,660


### seller_id下cat_id的个数

In [23]:
i4 = user_log_data[['seller_id','cat_id']].copy()
i4 = i4.drop_duplicates()
i4 = i4.groupby('seller_id').size().reset_index()
i4 = i4.rename(columns = {0:'seller_cat_count'})

In [64]:
i4.head()

,seller_id,seller_cat_count
0,1,44
1,2,10
2,3,4
3,4,7
4,5,23


### cat_id下brand_id计数

In [24]:
i5 = user_log_data[['brand_id','cat_id']].copy()
i5 = i5.drop_duplicates()
i5 = i5.groupby('cat_id').size().reset_index()
i5 = i5.rename(columns = {0:'cat_brand_count'})

In [121]:
i5[i5.cat_id.isnull()].compute()

,cat_id,cat_brand_count


## 处理user_info数据

### 按照user_id分类对gender编码

In [25]:
u2 = user_info[['user_id','gender']].copy()

In [69]:
len(u2)

424170

In [70]:
set(u2['user_id'].compute()).difference(set(u1.user_id.compute())) 
set(u1.user_id.compute()).difference(set(u2.user_id.compute()))
# 上下两个生成的结果都为为空集，再加上它们都有相同的行数42170，
# 所以可以断定两个数据集的user_id是相同的

set()

In [26]:
u2 = u2.fillna(value = {'gender':2.0}) # 因为数据介绍中也说了2.0 代表Unknown
# compute不能随便用，因为一旦用了的话就变成pandas和python中的数据存储格式了
# note that the compute() function returns in-memory results. 
# It converts Dask dataframes to Pandas dataframes, 
# Dask arrays to Numpy arrays, and Dask bags to lists

In [72]:
set(u2.gender.isnull().compute())

{False}

In [ ]:
u2 = u2.categorize(columns = ['gender'])
u2.gender

In [29]:
u2 = DummyEncoder().fit_transform(u2) 

### 对user_age_range编码
它不是连续值，所以还是用离散编码

In [30]:
ui1 = user_info[['user_id','age_range']].copy()
ui1['age_range'] = ui1.age_range.fillna(0.0)

In [31]:
ui1['age_range'] = ui1.age_range.map(lambda x: x if x != 8.0 else 7.0)

In [32]:
ui1 = ui1.drop_duplicates()
ui1 = ui1.categorize(columns = ['age_range']) 

In [33]:
ui1 = DummyEncoder().fit_transform(ui1) 

In [79]:
ui1.head()

,user_id,age_range_6.0,age_range_5.0,age_range_4.0,age_range_7.0,age_range_3.0,age_range_0.0,age_range_2.0,age_range_1.0
0,376517,1,0,0,0,0,0,0,0
1,234512,0,1,0,0,0,0,0,0
2,344532,0,1,0,0,0,0,0,0
3,186135,0,1,0,0,0,0,0,0
4,30230,0,1,0,0,0,0,0,0


## user_id、seller_id数据合并

In [34]:
# 将u1和ui1合并，u1：log数据按照user_id分类后计数，
# ui1：info数据按照user_id对age_range编码
u1ui1 = u1.merge(ui1,on = 'user_id') # 默认的how = inner

In [35]:
# 再将上面的u1ui1 和u3合并，u3：根据user_id对action_type编码
u1ui1u3 = u1ui1.merge(u3,on = 'user_id')

In [36]:
# 再将上面的和u2合并，u2：根据user_d对gender编码
u1ui1u3u2 = u1ui1u3.merge(u2,on = 'user_id')

In [37]:
# 再将上面和u4合并，u4：将user_id分类，对其操作的天数计数
u1ui1u3u2u4 = u1ui1u3u2.merge(u4,on = 'user_id')

In [38]:
# 将上面的结果和u7合并，u7：根据user_id对每个seller_id下的item_id的action_type编码、计数
u1ui1u3u2u4u7 = u1ui1u3u2u4.merge(u7,on = 'user_id')

In [39]:
# 将上面的结果和i3合并，i3：根据seller_id对其下的item_id计数
u1ui1u3u2u4u7i3 = u1ui1u3u2u4u7.merge(i3,on = 'seller_id')

In [42]:
u1ui1u3u2u4u7i3.columns

Index(['user_id', 'user_action_count', 'age_range_6.0', 'age_range_5.0',
       'age_range_4.0', 'age_range_7.0', 'age_range_3.0', 'age_range_0.0',
       'age_range_2.0', 'age_range_1.0', 'action_type_0', 'action_type_2',
       'action_type_3', 'action_type_1', 'gender_1.0', 'gender_0.0',
       'gender_2.0', 'user_handle_day_count', 'seller_id', 'item_id',
       'usi_action_type_0', 'usi_action_type_2', 'usi_action_type_3',
       'usi_action_type_1', 'seller_item_id_count'],
      dtype='object')

In [40]:
# 将上面的结果和i4合并，i4：根据seller_id对其下的cat_id计数
u1ui1u3u2u4u7i3i4 = u1ui1u3u2u4u7i3.merge(i4,on = 'seller_id')

In [41]:
u1ui1u3u2u4u7i3i4

,user_id,user_action_count,age_range_6.0,age_range_5.0,age_range_4.0,age_range_7.0,age_range_3.0,age_range_0.0,age_range_2.0,age_range_1.0,action_type_0,action_type_2,action_type_3,action_type_1,gender_1.0,gender_0.0,gender_2.0,user_handle_day_count,seller_id,item_id,usi_action_type_0,usi_action_type_2,usi_action_type_3,usi_action_type_1,seller_item_id_count,seller_cat_count
npartitions=30,,,,,,,,,,,,,,,,,,,,,,,,,,
,int64,int64,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,uint8,int64,int64,int64,uint8,uint8,uint8,uint8,int64,int64
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...


## train1集数据和user_seller数据合并 

In [43]:
usi = u1ui1u3u2u4u7i3i4.rename(columns={'seller_id':'merchant_id'})
# 因为在train1数据中的是merchant_id

In [126]:
usitrain1 = usi.merge(df_train1,on = ['user_id','merchant_id']) 

## 训练集模型测试

In [127]:
features = usitrain1.columns.difference(['user_id','merchant_id','label'])

In [49]:
usitrain1.columns

Index(['user_id', 'user_action_count', 'age_range_6.0', 'age_range_5.0',
       'age_range_4.0', 'age_range_7.0', 'age_range_3.0', 'age_range_0.0',
       'age_range_2.0', 'age_range_1.0', 'action_type_0', 'action_type_2',
       'action_type_3', 'action_type_1', 'gender_1.0', 'gender_0.0',
       'gender_2.0', 'user_handle_day_count', 'merchant_id', 'item_id',
       'usi_action_type_0', 'usi_action_type_2', 'usi_action_type_3',
       'usi_action_type_1', 'seller_item_id_count', 'seller_cat_count',
       'label'],
      dtype='object')

In [128]:
x = usitrain1[features.tolist()].fillna(value=0, method=None, limit=None, axis=0)
x = x.values

In [131]:
x

dask.array<values, shape=(nan, 24), dtype=int64, chunksize=(nan, 24)>

In [129]:
x.reshape((-1, 24))

ValueError: cannot convert float NaN to integer

In [110]:
type(x)

dask.array.core.Array

In [130]:
y = usitrain1['label'].fillna(value=0).values

In [132]:
y.reshape((-1, 1))

ValueError: cannot convert float NaN to integer

In [112]:
type(y)

dask.array.core.Array

In [115]:
lgr = LogisticRegression()

In [116]:
lgr_model = lgr.fit(x, y) 

NotImplementedError: Can not add intercept to array with unknown chunk shape

In [105]:
x, y = make_classification()

y

dask.array<lt, shape=(1000,), dtype=bool, chunksize=(100,)>

In [114]:
y_pre = lgr_model.predict(x)
accuracy_score(y,y_pre)

NameError: name 'lgr_model' is not defined

## 分类test中的数据

In [ ]:
usitest1 = usi.merge(df_test1,on = ['user_id','merchant_id'],how = 'right')

In [ ]:
x_test = usitest1[features]

In [ ]:
y_pre_test = lgr_model.predict(x_test)
prediction = df_test1.copy()
prediction['prob'] = y_pre_test

In [ ]:
prediciton.to_csv('prediction.csv',header=False,index=False)

至此即可将数据提交到比赛了。